In [71]:
#Define the data available for the model
metal_data = {
    "unit":"gram",
    "currency": "USD",
    "prices": {
        "gold": 88.1553,
        "silver": 1.0523,
        "platinum": 32.169,
        "palladium": 35.8252,
        "copper": 0.0098,
        "aluminum": 0.0026,
        "lead": 0.0021,
        "nickel": 0.0159,
        "zinc": 0.0031,
    }
}

In [72]:
from langchain_core.tools import tool
import requests

# Define the tools for the agent to use, it is necessary to specify that each function is a tool
@tool
def get_metal_price(metal_name: str) -> str:
    """Fetches the current per gram in USD price of the specified metal.

    Args:
        metal_name : The name of the metal (e.g., 'gold', 'silver', 'platinum').

    Returns:
        float: The current price of the metal in dollars per gram.

    Raises:
        KeyError: If the specified metal is not found in the data source.
    """
    try:
        metal_name = metal_name.lower().strip()
        prices = metal_data["prices"]
        currency = metal_data["currency"]
        unit=metal_data["unit"]
        if metal_name not in prices:
            raise KeyError(
                f"Metal {metal_name} not found. Available metals: {', '.join(metal_data['prices'].keys())}"
            )
        price=prices[metal_name]
        return f"The current price of {metal_name} is {price} {currency} per {unit}."
    except Exception as e:
        raise Exception(f"Error fetching metal price: {str(e)}")
    
@tool    
def get_currency_exchange(base: str, target: str) -> str:
    """
    Returns the exchange rate from base currency to target currency.

    Args:
        base (str): The base currency (e.g., 'USD').
        target (str): The target currency (e.g., 'EUR').

    Returns:
        str: A human-readable string showing the exchange rate,
             or an error message if the pair is not found.
    """
    fake_rates = {
        ("usd", "eur"): 0.8,
        ("eur", "usd"): 1.8,
        ("usd", "gbp"): 0.7
    }
    rate = fake_rates.get((base.lower(), target.lower()))
    if rate is None:
        return f"No exchange rate found for {base.upper()} to {target.upper()}"
    return  f"{base.upper()} = {rate} {target.upper()}"

@tool
def search_wikipedia(query: str) -> str:
    """Search Wikipedia for a summary related to the query."""
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{query.replace(' ', '_')}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get("extract", "No summary found.")
    else:
        return "No relevant results found."

"""@tool
def search_duckduckgo(query: str) -> str:
    
    #Performs a DuckDuckGo search using a JSON endpoint and returns the first result snippet.
    
    try:
        url = "https://api.duckduckgo.com/"
        params = {"q": query, "format": "json", "no_html": 1, "skip_disambig": 1}
        resp = requests.get(url, params=params, timeout=10)
        data = resp.json()

        if "AbstractText" in data and data["AbstractText"]:
            return data["AbstractText"]
        elif data.get("RelatedTopics"):
            for topic in data["RelatedTopics"]:
                if isinstance(topic, dict) and topic.get("Text"):
                    return topic["Text"]
        return "No relevant results found."
    except Exception as e:
        return f"Search error: {e}"
import requests"""


'@tool\ndef search_duckduckgo(query: str) -> str:\n\n    #Performs a DuckDuckGo search using a JSON endpoint and returns the first result snippet.\n\n    try:\n        url = "https://api.duckduckgo.com/"\n        params = {"q": query, "format": "json", "no_html": 1, "skip_disambig": 1}\n        resp = requests.get(url, params=params, timeout=10)\n        data = resp.json()\n\n        if "AbstractText" in data and data["AbstractText"]:\n            return data["AbstractText"]\n        elif data.get("RelatedTopics"):\n            for topic in data["RelatedTopics"]:\n                if isinstance(topic, dict) and topic.get("Text"):\n                    return topic["Text"]\n        return "No relevant results found."\n    except Exception as e:\n        return f"Search error: {e}"\nimport requests'

En este bloque definimos la herramienta get_metal_price con el decorador @tool, igual que en el ejemplo de RAGAS
docs.ragas.io
. Esta función recibe un nombre de metal y devuelve su precio simulado.

In [73]:
from langchain_ollama.chat_models import ChatOllama
from langgraph.prebuilt import create_react_agent

# Instanciamos el modelo LLM local usando Ollama (Llama 3.2)
llm = ChatOllama(
    model="llama3.2",   # Usamos el modelo Llama 3.2 local
    temperature=0
)

# Vinculamos nuestra herramienta al LLM
tools = [get_metal_price,get_currency_exchange,search_wikipedia]
llm_with_tools = llm.bind_tools(tools)

agent = create_react_agent(
    model=llm_with_tools,
    tools=[get_metal_price, get_currency_exchange,search_wikipedia],
    prompt="""
You are a ReAct agent.

For questions about metal prices like "What is the price of METAL in CUR?":
1) Call `get_metal_price` with the metal name (e.g., "gold").
2) Then call `get_currency_exchange` with base='USD' and target=CUR.
3) Combine both results in a final response with the price converted.

For general knowledge or factual questions (e.g., "Who is the president of France?"):
1) Call `search_wikipedia` with a well-formed query.
2) Use the result to respond clearly and concisely.


When you call a tool, your final response must be exactly and only the content returned by the tool. Do not rephrase, summarize or add any additional information. Do not use your own knowledge.

"""
)

Aquí creamos el objeto ChatOllama de LangChain usando el modelo Llama3.2 local python.langchain.com, y vinculamos la lista de herramientas (sólo get_metal_price) con .bind_tools python.langchain.com. Esto permite que el modelo invoque la herramienta cuando sea necesario.

In [74]:
from langgraph.prebuilt import ToolNode
from langgraph.graph import END
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

# 1) State
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# 2) Nodes
tool_functions = [get_metal_price,get_currency_exchange,search_wikipedia]
tool_node = ToolNode(tool_functions)

def assistant(state: GraphState):
    # 1) Pasar todo el historial al agente ReAct
    result = agent.invoke({"messages": state["messages"]})
    
    # 2) Extraer la lista de nuevos mensajes
    new_msgs = result["messages"]
    
    # 3) Devolver el historial completo concatenado
    return {"messages": state["messages"] + new_msgs}

def should_continue(state: GraphState):
    last = state["messages"][-1]

    # 1) Si es un dict y contiene tool_calls
    if isinstance(last, dict) and last.get("tool_calls"):
        return "tools"

    # 2) Si es un objeto con atributo tool_calls (p.ej. AIMessage)
    tc = getattr(last, "tool_calls", None)
    if tc:
        return "tools"

    return END

In [75]:
from langgraph.graph import START, StateGraph
from IPython.display import Image, display

# Define a new graph for the agent
builder = StateGraph(GraphState)

# Define the two nodes we will cycle between
builder.add_node("assistant", assistant)
builder.add_node("tools", tool_node)

# Set the entrypoint as `agent`
builder.add_edge(START, "assistant")

# Making a conditional edge
# should_continue will determine which node is called next.
builder.add_conditional_edges("assistant", should_continue, {"tools", END})

# Making a normal edge from `tools` to `agent`.
builder.add_edge("tools", "assistant")

# Compile and display the graph for a visual overview
react_graph = builder.compile()
#display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [76]:
from langchain.schema.messages import AIMessage,ToolMessage,HumanMessage
from copy import deepcopy
import json
import uuid

def normalize_args(args):
    return {k.lower(): str(v).lower() for k, v in args.items()}

def fix_tool_calls_for_openai_format(messages):
    final_messages = []
    tool_message_buffer = {}
    used_tool_call_ids = set()

    # Indexar ToolMessages por tool_call_id
    for msg in messages:
        if isinstance(msg, ToolMessage):
            tool_message_buffer[msg.tool_call_id] = msg

    for msg in messages:
        if isinstance(msg, HumanMessage):
            final_messages.append(msg)

        elif isinstance(msg, AIMessage) and msg.tool_calls and len(msg.tool_calls) > 1:
            for tool_call in msg.tool_calls:
                # Normalizar los args aquí
                norm_args = normalize_args(tool_call["args"])

                new_msg = deepcopy(msg)
                new_msg.tool_calls = [{
                    "name": tool_call["name"],
                    "args": norm_args,
                    "id": tool_call.get("id", f"call_{uuid.uuid4().hex[:24]}"),
                    "type": "tool_call"
                }]
                new_msg.additional_kwargs["tool_calls"] = [{
                    "id": tool_call.get("id", f"call_{uuid.uuid4().hex[:24]}"),
                    "type": "function",
                    "function": {
                        "name": tool_call["name"],
                        "arguments": json.dumps(norm_args)
                    }
                }]
                final_messages.append(new_msg)

                tool_msg = tool_message_buffer.get(tool_call["id"])
                if tool_msg:
                    final_messages.append(tool_msg)
                    used_tool_call_ids.add(tool_call["id"])

        elif isinstance(msg, AIMessage) and msg.tool_calls:
            tool_call = msg.tool_calls[0]
            norm_args = normalize_args(tool_call["args"])

            msg.tool_calls[0]["args"] = norm_args
            msg.additional_kwargs["tool_calls"] = [{
                "id": tool_call.get("id", f"call_{uuid.uuid4().hex[:24]}"),
                "type": "function",
                "function": {
                    "name": tool_call["name"],
                    "arguments": json.dumps(norm_args)
                }
            }]
            final_messages.append(msg)

        elif isinstance(msg, AIMessage):
            final_messages.append(msg)

        elif isinstance(msg, ToolMessage):
            if msg.tool_call_id not in used_tool_call_ids:
                final_messages.append(msg)

    return final_messages


In [77]:
# Ejemplo de ejecución del agente con una pregunta del usuario
"""messages = {"messages":[{"role":"user","content":"What is the price of gold in EUR?"}]}
result = react_graph.invoke(messages)

# Mostrar el historial de mensajes resultante
for msg in result["messages"]:
    print(msg)"""

'messages = {"messages":[{"role":"user","content":"What is the price of gold in EUR?"}]}\nresult = react_graph.invoke(messages)\n\n# Mostrar el historial de mensajes resultante\nfor msg in result["messages"]:\n    print(msg)'

In [78]:
#fixed_messages=fix_tool_calls_for_openai_format(result["messages"])
#fixed_messages

In [81]:
# Ejemplo de ejecución del agente con una pregunta del usuario
messages = {"messages":[{"role":"user","content":"What is the name of the president of France?"}]}
result = react_graph.invoke(messages)

# Mostrar el historial de mensajes resultante
for msg in result["messages"]:
    print(msg)

content='What is the name of the president of France?' additional_kwargs={} response_metadata={} id='6ba22744-eba8-4665-b9a7-9f514835a1f4'
content='' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-05-28T18:14:58.6696809Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11027147300, 'load_duration': 24875900, 'prompt_eval_count': 581, 'prompt_eval_duration': 9491806600, 'eval_count': 20, 'eval_duration': 1507745600, 'model_name': 'llama3.2'} id='run--56702508-ddce-490a-992e-a513fdc5b5a1-0' tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'President of France'}, 'id': '20dff9e0-e8d5-4bc1-9de6-b634d67f0206', 'type': 'tool_call'}] usage_metadata={'input_tokens': 581, 'output_tokens': 20, 'total_tokens': 601}
content='The president of France, officially the president of the French Republic, is the executive head of state of France, and the commander-in-chief of the French Armed Forces. As the presidency is the supreme magistracy of the cou

In [80]:
fixed_messages=fix_tool_calls_for_openai_format(result["messages"])
fixed_messages

[HumanMessage(content='Who is the president of France?', additional_kwargs={}, response_metadata={}, id='9a964459-3796-46fd-a1f4-b287dbb17f13'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '55ba4744-cc9d-4bab-acfe-dbf106c4d3c0', 'type': 'function', 'function': {'name': 'search_wikipedia', 'arguments': '{"query": "president of france"}'}}]}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-28T18:13:57.5508677Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12944868300, 'load_duration': 23603500, 'prompt_eval_count': 578, 'prompt_eval_duration': 11427244100, 'eval_count': 20, 'eval_duration': 1492437400, 'model_name': 'llama3.2'}, id='run--9871165b-d22e-43d1-a987-ded1dad772f0-0', tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'president of france'}, 'id': '55ba4744-cc9d-4bab-acfe-dbf106c4d3c0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 578, 'output_tokens': 20, 'total_tokens': 598}),
 ToolMessage(content='The president 